<a href="https://colab.research.google.com/github/GBH7/Meachine-Learning/blob/main/DeepLearning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1번.

#import modules
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Data Preparation
transf = transforms.Compose([transforms.ToTensor()])

trainset = dset.CIFAR10(root='./data',train=True,download=True,transform=transf)
testset = dset.CIFAR10(root='./data',train=False,download=True,transform=transf)

batch_size = 32
learning_rate = 0.0001
num_epoch = 50

#합성곱 연산 2번 하는 함수
def conv_2_block(in_dim,out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

def conv_3_block(in_dim,out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

def conv_4_block(in_dim,out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
#Model_1 architecture

class VGG16(nn.Module):
  def __init__(self):
    super(VGG16,self).__init__()
    self.feature = nn.Sequential(
        conv_2_block(3,64),
        conv_2_block(64,128),
        conv_3_block(128,256),
        conv_3_block(256,512),
        conv_3_block(512,512),
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(512,100),
        nn.ReLU(True),
        nn.Linear(100,20),
        nn.ReLU(True),
        nn.Linear(20,10)
    )

  def forward(self,x):
    x = self.feature(x)
    x = x.view(x.size(0),-1)
    x = self.fc_layer(x)
    return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = VGG16().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
trainloader = DataLoader(trainset, batch_size=32)
testloader = DataLoader(testset,batch_size=32)

In [ ]:
for i in range(num_epoch):
  for j,[image,label] in enumerate(trainloader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()
  
  if i % 10 == 0:
    print(loss)

tensor(2.3572, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1395, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0123, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0132, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0169, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
#Model_1 Evaluation
correct = 0
total = 0
with torch.no_grad():
  model.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test accuracy: %.2f %%'%(100*correct/total))

Test accuracy: 77.54 %


In [ ]:
#Model_2 architecture

class VGG19(nn.Module):
  def __init__(self):
    super(VGG19,self).__init__()
    self.feature = nn.Sequential(
        conv_2_block(3,64),
        conv_2_block(64,128),
        conv_4_block(128,256),
        conv_4_block(256,512),
        conv_4_block(512,512),
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(512,100),
        nn.ReLU(True),
        nn.Linear(100,20),
        nn.ReLU(True),
        nn.Linear(20,10)
    )

  def forward(self,x):
    x = self.feature(x)
    x = x.view(x.size(0),-1)
    x = self.fc_layer(x)
    return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = VGG19().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

for i in range(num_epoch):
  for j,[image,label] in enumerate(trainloader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()
  
  if i % 10 == 0:
    print(loss)

#Model_2 Evaluation
correct = 0
total = 0
with torch.no_grad():
  model.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test accuracy: %.2f %%'%(100*correct/total))

tensor(2.1956, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6520, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1441, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0227, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0010, device='cuda:0', grad_fn=<NllLossBackward0>)
Test accuracy: 77.92 %


In [ ]:
#Model_3 architecture

class VGG13(nn.Module):
  def __init__(self):
    super(VGG13,self).__init__()
    self.feature = nn.Sequential(
        conv_2_block(3,64),
        conv_2_block(64,128),
        conv_2_block(128,256),
        conv_2_block(256,512),
        conv_2_block(512,512),
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(512,100),
        nn.ReLU(True),
        nn.Linear(100,20),
        nn.ReLU(True),
        nn.Linear(20,10)
    )

  def forward(self,x):
    x = self.feature(x)
    x = x.view(x.size(0),-1)
    x = self.fc_layer(x)
    return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = VGG13().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

for i in range(num_epoch):
  for j,[image,label] in enumerate(trainloader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()
  
  if i % 10 == 0:
    print(loss)

#Model_3 Evaluation
correct = 0
total = 0
with torch.no_grad():
  model.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test accuracy: %.2f %%'%(100*correct/total))

tensor(1.9988, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2293, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0059, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0102, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0061, device='cuda:0', grad_fn=<NllLossBackward0>)
Test accuracy: 78.18 %


In [ ]:
torch.save(model,'model.pt')

최종 성능 78.18%로 모델 3(합성곱 층 10개)에서 가장 높은 성능을 기록함.

사전 훈련된 구현 모델을 이용해서 하려 했으나 colab에서 이용량이 계속 초과되어 자동 종료되는 바람에 사전훈련 모델을 사용하지는 못했습니다.

In [ ]:
#2번

#import modules
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Data Preparation
transf = transforms.Compose([
                             transforms.Resize(32),
                             transforms.ToTensor(),
                             ])

trainset = dset.STL10(root='./data',split='train',download=True,transform=transf)
testset = dset.STL10(root='./data',split='test',download=True,transform=transf)

batch_size = 32
learning_rate = 0.0001
num_epoch = 50

trainloader = DataLoader(trainset, batch_size=32)
testloader = DataLoader(testset, batch_size=32)

def conv_2_block(in_dim,out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model


def conv_3_block(in_dim,out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

def conv_4_block(in_dim,out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


In [ ]:
#Model_1 architecture

class VGG16(nn.Module):
  def __init__(self):
    super(VGG16,self).__init__()
    self.feature = nn.Sequential(
        conv_2_block(3,64),
        conv_2_block(64,128),
        conv_3_block(128,256),
        conv_3_block(256,512),
        conv_3_block(512,512),
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(512,100),
        nn.ReLU(True),
        nn.Linear(100,20),
        nn.ReLU(True),
        nn.Linear(20,10)
    )

  def forward(self,x):
    x = self.feature(x)
    x = x.view(x.size(0),-1)
    x = self.fc_layer(x)
    return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = VGG16().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
for i in range(num_epoch):
  for j,[image,label] in enumerate(trainloader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()
  
  if i % 10 == 0:
    print(loss)

#Model_1 Evaluation
correct = 0
total = 0
with torch.no_grad():
  model.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test accuracy: %.2f %%'%(100*correct/total))

tensor(1.0526, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.9179, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4233, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1617, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0393, device='cuda:0', grad_fn=<NllLossBackward0>)
Test accuracy: 45.21 %


In [ ]:
#model1 evaluation

class VGG19(nn.Module):
  def __init__(self):
    super(VGG19,self).__init__()
    self.feature = nn.Sequential(
        conv_2_block(3,64),
        conv_2_block(64,128),
        conv_4_block(128,256),
        conv_4_block(256,512),
        conv_4_block(512,512),
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(512,100),
        nn.ReLU(True),
        nn.Linear(100,20),
        nn.ReLU(True),
        nn.Linear(20,10)
    )

  def forward(self,x):
    x = self.feature(x)
    x = x.view(x.size(0),-1)
    x = self.fc_layer(x)
    return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = VGG19().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

for i in range(num_epoch):
  for j,[image,label] in enumerate(trainloader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()
  
  if i % 10 == 0:
    print(loss)

#Model_2 Evaluation
correct = 0
total = 0
with torch.no_grad():
  model.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test accuracy: %.2f %%'%(100*correct/total))

tensor(2.3110, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3007, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.8031, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.7406, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4699, device='cuda:0', grad_fn=<NllLossBackward0>)
Test accuracy: 38.08 %


In [ ]:
#Model_3 architecture

class VGGself(nn.Module):
  def __init__(self):
    super(VGGself,self).__init__()
    self.feature = nn.Sequential(
        conv_2_block(3,64),
        conv_3_block(64,128),
        conv_3_block(128,256),
        conv_3_block(256,512),
        conv_4_block(512,512),
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(512,100),
        nn.ReLU(True),
        nn.Linear(100,20),
        nn.ReLU(True),
        nn.Linear(20,10)
    )

  def forward(self,x):
    x = self.feature(x)
    x = x.view(x.size(0),-1)
    x = self.fc_layer(x)
    return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = VGGself().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

for i in range(num_epoch):
  for j,[image,label] in enumerate(trainloader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()
  
  if i % 10 == 0: 
    print(loss)

#Model_3 Evaluation
correct = 0
total = 0
with torch.no_grad():
  model.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test accuracy: %.2f %%'%(100*correct/total))

tensor(2.2865, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2849, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.8200, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6949, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.2757, device='cuda:0', grad_fn=<NllLossBackward0>)
Test accuracy: 39.60 %


최종 성능 45.21%로 모델 1(합성곱 층 13개)에서 가장 높은 성능을 기록함